# Tagger 

This example shows the definition of a *tagger model* with a _multi-feature CNN token-to-vector encoder_.

It works like this:

* `MultiEmbed` layer: multiple numeric ID features are extracted for each word, and each feature is separately embedded. The separate vectors are concatenated and returned.

* `Hidden`: the concatenated embeddings are passed through a dense layer with a “maxout” activation [Goodfellow et al, 2013](https://arxiv.org/abs/1302.4389).

* `CNN layers`: several convolutional layers are applied in sequence for contextual encoding (`clone` function). Each CNN layer performs a “sequence-to-column” transformation, where a window of surrounding words is concatenated to each vector (`expand_window`). A `Hidden` layer then maps the result back to the original dimensionality. Residual connections and layer normalization are used to assist convergence (`residual` - a unary combinator creating a residual connection).

* `Softmax`: gives the most likelly tag as the output of the model.

In [1]:
from thinc.api import prefer_gpu
prefer_gpu()

from thinc.api import fix_random_seed
fix_random_seed(0)

from tqdm import tqdm



In [2]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_y), (test_X, test_y) = data
    model.initialize(X=train_X[:5], Y=train_y[:5])
    for n in range(n_iter):
        loss = 0.0
        batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            d_loss = []
            for i in range(len(Yh)):
                d_loss.append(Yh[i] - Y[i])
                loss += ((Yh[i] - Y[i]) ** 2).sum()
            backprop(d_loss)
            model.finish_update(optimizer)
        score = evaluate(model, test_X, test_y, batch_size)
        print(f"{n}\t{loss:.2f}\t{score:.3f}")
        
def evaluate(model, test_X, test_Y, batch_size):
    correct = 0
    total = 0
    for X, Y in model.ops.multibatch(batch_size, test_X, test_Y):
        Yh = model.predict(X)
        for yh, y in zip(Yh, Y):
            correct += (y.argmax(axis=1) == yh.argmax(axis=1)).sum()
            total += y.shape[0]
    return float(correct / total)

In [3]:
from thinc.api import HashEmbed, Maxout, Softmax, expand_window, Relu
from thinc.api import residual, strings2arrays, with_array, clone, chain, concatenate

width = 128
depth = 4
n_tags = 17

def MultiEmbed(width):
    return concatenate(
        HashEmbed(width, 4000, column=0),
        HashEmbed(width // 2, 2000, column=0),
        HashEmbed(width // 2, 2000, column=0),
        HashEmbed(width // 2, 2000, column=0),
    )

def Hidden(nO, dropout=0.2):
     return Maxout(nO, nP=3, normalize=True, dropout=dropout)

def CNN(width):
    return residual(chain(expand_window(1), Hidden(width)))

model = chain(
    strings2arrays(),
    with_array(
        chain(
            MultiEmbed(width),
            Hidden(width),
            clone(CNN(width), depth),
            Softmax(n_tags)
        )
    )
)

# model.to_dict()


In [4]:
import ml_datasets


CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""


from thinc.api import registry, Config

config = Config().from_str(CONFIG)
loaded_config = registry.make_from_config(config)

optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

(train_X, train_y), (test_X, test_y) = data = ml_datasets.ud_ancora_pos_tags()



In [5]:
train_model(data, model, optimizer, n_iter, batch_size)

  0%|          | 0/112 [00:00<?, ?it/s]          

0	111415.20	0.922


  0%|          | 0/112 [00:00<?, ?it/s]          

1	35114.85	0.941


  0%|          | 0/112 [00:00<?, ?it/s]          

...

  0%|          | 0/112 [00:00<?, ?it/s]          

7	8598.47	0.952


  0%|          | 0/112 [00:00<?, ?it/s]          

8	7719.94	0.951


9	7155.96	0.950
